In [ ]:
from multiprocessing.pool import Pool

from ddosdb.file_type_parser import *
from ddosdb.dataframe_analysis import *
from ddosdb.attack_vector_anonymizer import *

In [2]:
def anonymize(_input_file, _file_type, _victim_ip, _fingerprint):
    return anonymize_attack_vector(_input_file, _file_type, _victim_ip, _fingerprint)


def ddos_dissector(input_file):
    print('1. Analysing the type of input file (e.g., pcap, pcapng, nfdump, netflow, and ipfix)...') 
    file_type = determine_file_type(input_file)
    
    print('2. Converting input file to dataframe...') 
    df = convert_to_dataframe(input_file, file_type) 
    
    print('3. Analysing the dataframe for finding attack patterns...')
    victim_ip, fingerprints = analyze_dataframe(df, file_type)

    print('4. Creating annonymized files containing only the attack vectors...\n')

    with Pool(len(fingerprints)) as p:
        # Run all fingerprints at the same time
        items = [(input_file, file_type, victim_ip, x) for x in fingerprints]
        p.starmap(anonymize, items)

    print('\n\nDONE!!!!!')

In [4]:
# FOR TESTING PURPOSES
input_file = "input4test/1.pcap"
ddos_dissector(input_file)


1. Analysing the type of input file (e.g., pcap, pcapng, nfdump, netflow, and ipfix)...

2. Converting input file to dataframe...

3. Analysing the dataframe for finding attack patterns...

DISTRIBUTION OF DESTINATION IPs:
 145.220.192.4    1913687
Name: _ws.col.Destination, dtype: int64

DISTRIBUTION OF PROTOCOLS:
 IPv4       820263
DNS        712241
ICMP       368100
UDP         12599
QUIC          319
TCP            58
DAYTIME        20
XYPLEX         19
NBDS           17
XTACACS        14
TIME            8
ECHO            8
TFTP            7
ISAKMP          5
Chargen         3
RakNet          2
NBNS            2
SNMP            1
SSDP            1
Name: _ws.col.Protocol, dtype: int64

FRAGMENTATION DISTRIBUTION:
 True    820263
Name: fragmentation, dtype: int64

ATTACK VECTOR 1: (['_ws.col.Protocol']=='IPv4')&(['fragmentation']==True)
  - Packets:820263
  - #Src_IPs:41837

DISTRIBUTION OF PROTOCOLS:
 DNS        712241
ICMP       368100
UDP         12599
QUIC          319
TCP      


INTERSECTION OF SOURCE IPS IN ATTACK VECTORS:
          1        2       3        4      5    6     7     8
1  41837.0  27266.0    27.0  23017.0   12.0  4.0   3.0   1.0
2  27266.0  30551.0    19.0   9809.0    9.0  0.0   0.0   0.0
3     27.0     19.0  5214.0     22.0    4.0  0.0   0.0   0.0
4  23017.0   9809.0    22.0  25027.0   12.0  1.0   0.0   0.0
5     12.0      9.0     4.0     12.0  270.0  0.0   0.0   0.0
6      4.0      0.0     0.0      1.0    0.0  4.0   0.0   0.0
7      3.0      0.0     0.0      0.0    0.0  0.0  12.0   0.0
8      1.0      0.0     0.0      0.0    0.0  0.0   0.0  12.0

4. Creating annonymized files containing only the attack vectors...



DONE!!!!!


In [4]:
if __name__ == '__main__':
    import argparse
    import os.path

    parser = argparse.ArgumentParser(description='')

    parser.add_argument('--input', metavar='input_file', required=True,
                        help='Path of a input file')

    # parser.add_argument('--debug', metavar='debug', required=False, default=False,
    #                     help='enable debug')

    args = parser.parse_args()
    input_file = args.input
    # debug = args.debug

    if os.path.isfile(input_file):
        ddos_dissector(input_file)
    else:
        print("We were unable to find the file. Please check the file path!")

usage: ipykernel_launcher.py [-h] --input input_file [--debug debug]
ipykernel_launcher.py: error: the following arguments are required: --input


SystemExit: 2